<a href="https://colab.research.google.com/github/sagar9926/EPAI_session16/blob/main/EPAI_session_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 1 :

The files are:

- personal_info.csv - personal information such as name, gender, etc. (one row per person)

- vehicles.csv - what vehicle people own (one row per person)

- employment.csv - where a person is employed (one row per person)

- update_status.csv - when the person's data was created and last updated

Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

In [1]:
!git clone https://github.com/sagar9926/EPAI_session16.git

fatal: destination path 'EPAI_session16' already exists and is not an empty directory.


In [2]:
cd EPAI_session16

/content/EPAI_session16


In [3]:
!ls

employment.csv	EPAI_session_16.ipynb  README.md	  vehicles.csv
EPAI_session16	personal_info.csv      update_status.csv


In [4]:
from collections import namedtuple
import datetime
import csv

In [5]:
def read_file(file_name):
  with open(file_name) as f:
    rows = csv.reader(f,delimiter = ',',quotechar = '"')
    yield from rows


In [6]:
data_type_dict = {
    "employment.csv" : [str,str,str,str],
    'personal_info.csv' : [str,str,str,str,str],
    'update_status.csv' : [str,datetime.datetime.strptime,datetime.datetime.strptime],
    'vehicles.csv' : [str,	str,	str,	int]

}
date_format = '%Y-%m-%dT%H:%M:%SZ'

In [7]:
class file_iter:
  def __init__(self,file_name):
    self.file_name = file_name
    self.data_type = data_type_dict[self.file_name]
    self.file = read_file(self.file_name)
    self.columns = " ".join(next(self.file))
    self.row_name_tuple = namedtuple((self.file_name[0].upper() + self.file_name[1:-4]),self.columns)
    self.row_index = 0
  
  def __iter__(self):
    for row in self.file:
      if self.file_name != 'update_status.csv':
        row = [type_(data) for type_ , data in zip(self.data_type,row)]
      else :
        row = [type_(data) if type_ == str else type_(data,date_format) for type_ , data in zip(self.data_type,row)]
      yield self.row_name_tuple(*row)



### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

In [8]:
employment_iter = file_iter('employment.csv')
personal_info_iter = file_iter('personal_info.csv')
update_status_iter = file_iter('update_status.csv')
vehicles_iter = file_iter('vehicles.csv')

In [9]:
count = 0
for x in personal_info_iter: 
  print(x)
  count += 1
  if count == 3:
    break

Personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Personal_info(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Personal_info(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')


In [10]:
count = 0
for x in update_status_iter: 
  print(x)
  count += 1
  if count == 3:
    break

Update_status(ssn='100-53-9824', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
Update_status(ssn='101-71-4702', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57))
Update_status(ssn='101-84-0356', last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), created=datetime.datetime(2016, 9, 21, 23, 4, 7))


In [11]:
count = 0
for x in vehicles_iter: 
  print(x)
  count += 1
  if count == 3:
    break

Vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
Vehicles(ssn='101-71-4702', vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
Vehicles(ssn='101-84-0356', vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)


In [19]:
qc = file_iter('employment.csv')

In [20]:
qc

In [22]:
next(iter(qc))

Employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')

## Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns. Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

In [12]:
employment_iter = file_iter('employment.csv')
personal_info_iter = file_iter('personal_info.csv')
update_status_iter = file_iter('update_status.csv')
vehicles_iter = file_iter('vehicles.csv')

In [27]:
class AllData :
  def __init__(self,employment,personal_info,update_status,vehicles):
    self.employment_fname = employment
    self.personal_info_fname = personal_info
    self.update_status_fname = update_status
    self.vehicles_fname = vehicles

  def __iter__(self):
    return self.AllDataIterator(self.employment_fname,self.personal_info_fname,self.update_status_fname,self.vehicles_fname)

  class AllDataIterator:
    def __init__(self,employment,personal_info,update_status,vehicles):
      print("employment , ",employment)
      self.employment_iter = file_iter(employment)
      self.personal_info_iter = file_iter(personal_info)
      self.update_status_iter = file_iter(update_status)
      self.vehicles_iter = file_iter(vehicles)
      self.row_index = 0

    def __iter__(self):
      return self

    def __next__(self):
      if self.row_index >= 1000:
        raise StopIteration
      else:
        self.row_index += 1
        for employment , personal_info , update_status , vehicles in zip(self.employment_iter,self.personal_info_iter,self.update_status_iter,self.vehicles_iter):
          employment_dict = dict(employment._asdict()) 
          personal_info_dict = dict(personal_info._asdict())
          update_status_dict = dict(update_status._asdict())
          vehicles_dict = dict(vehicles._asdict())

          combined_dict = {**employment_dict , **personal_info_dict , **update_status_dict , **vehicles_dict }
          ALL_Data = namedtuple('ALL_Data',combined_dict)

          return ALL_Data(**combined_dict)
          
      

In [28]:
qc = AllData('employment.csv','personal_info.csv','update_status.csv','vehicles.csv')

In [29]:
for i in qc:
  print(i)

employment ,  employment.csv
ALL_Data(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
ALL_Data(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
ALL_Data(employer='Connelly Group', department='Research and Development', employee_id='98-7952860', ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', last_updated=datetime.datetime(2017, 10, 

In [31]:
for i in qc:
  print(i)

employment ,  employment.csv
ALL_Data(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
ALL_Data(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
ALL_Data(employer='Connelly Group', department='Research and Development', employee_id='98-7952860', ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', last_updated=datetime.datetime(2017, 10, 